In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

# Load environment variables
os.environ["WEAVIATE_API_KEY"] = os.getenv("WEAVIATE_API_KEY")
os.environ["WEAVIATE_CLUSTER_URL"] = os.getenv("WEAVIATE_CLUSTER_URL")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")


In [3]:
import weaviate
from weaviate.classes.init import Auth

cluster_url = os.environ.get("WEAVIATE_CLUSTER_URL", "http://localhost:8080")  # We can host Weaviate locally using Docker
api_key = os.environ.get("WEAVIATE_API_KEY")

if api_key:
    client = weaviate.connect_to_weaviate_cloud(
        cluster_url=cluster_url,
        auth_credentials=Auth.api_key(api_key)
    )
else:
    client = weaviate.Client(cluster_url)  # Local OSS without auth
client


In [4]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)

In [5]:
from langchain_weaviate.vectorstores import WeaviateVectorStore

vector_db = WeaviateVectorStore(
    client=client,
    index_name="RAG",
    text_key="content",
    embedding=embeddings,
)
vector_db

In [12]:
from langchain_core.documents import Document

documents = [
    Document(page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.", metadata={"source": "tweet"}),
    Document(page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.", metadata={"source": "news"}),
    Document(page_content="Building an exciting new project with LangChain - come check it out!", metadata={"source": "tweet"}),
    Document(page_content="Robbers broke into the city bank and stole $1 million in cash.", metadata={"source": "news"}),
    Document(page_content="Wow! That was an amazing movie. I can't wait to see it again.", metadata={"source": "tweet"}),
    Document(page_content="Is the new iPhone worth the price? Read this review to find out.", metadata={"source": "website"}),
    Document(page_content="The top 10 soccer players in the world right now.", metadata={"source": "website"}),
    Document(page_content="LangGraph is the best framework for building stateful, agentic applications!", metadata={"source": "tweet"}),
    Document(page_content="The stock market is down 500 points today due to fears of a recession.", metadata={"source": "news"}),
    Document(page_content="I have a bad feeling I am going to get deleted :(", metadata={"source": "tweet"})
]

vector_db.add_documents(documents=documents)

['9b3a1883-f328-4d78-8376-27bcfbe23984',
 'a76cc67f-2469-4fe0-b183-709e8f7ec707',
 'd700b6d5-8229-4cd4-8057-394bbcf2bee5',
 'a18ac6e9-258e-436e-aa8b-d433651fb1b1',
 'a54fcacf-3855-44aa-9a93-985eafe03f9c',
 '70ab853e-c216-4d07-8b1d-5abc232f2f5b',
 '6605b3da-8e80-4720-8bf7-d9bf3a4cf759',
 '252adda2-3473-4a62-b274-211bc5857a52',
 'dc6f4478-9312-4052-9c60-ee77eee3b048',
 'a4a13d7b-fdbf-4228-b995-24f103caa660']

In [ ]:
# client.collections.delete("RAG")  # drops entire collection


In [16]:
results = vector_db.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
)
results

[Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [8]:
### Retriever 
retriever = vector_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.4},
)
retriever.invoke("LangChain provides abstractions to make working with LLMs easy")

[Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again.")]

In [11]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="openai/gpt-oss-20b")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x121b2da30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x121b2ffb0>, model_name='openai/gpt-oss-20b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [12]:
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [13]:

from langchain_core.prompts import ChatPromptTemplate
## Create a prompt template
system_prompt="""You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.

Context: {context}"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

In [14]:
### Create a document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nUse three sentences maximum and keep the answer concise.\n\nContext: {context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x121b2da30>, async_client=<groq.resources.chat.completions.A

In [15]:
### Create The Final RAG Chain
from langchain.chains import create_retrieval_chain
rag_chain=create_retrieval_chain(retriever,document_chain)
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['WeaviateVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_weaviate.vectorstores.WeaviateVectorStore object at 0x1127cbbc0>, search_type='similarity_score_threshold', search_kwargs={'k': 3, 'score_threshold': 0.4}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. \n

In [19]:
response = rag_chain.invoke({"input": "What I had for breakfast?"})
response["answer"]

'You had chocolate chip pancakes and scrambled eggs for breakfast.'